# 1. Introduction

In this mission, we're going to continue to practice constructing complex joins, while also learning how to:

* Build and format your queries for readability
* Creating named subqueries and views
* Combining data using set operations.

# 2. Writing Readable Queries

 Taking the time to write your queries to be more easily understood will take a little extra time now, but will save you time when you come back to old queries that you have written, and help your colleagues when you're working in a data team.

One obvious area when it comes to writing queries is the use of **capitalization and whitespace**. Because white space doesn't have any meaning in SQL, it can be used to help convey meaning in a complex query.

**As you can see, a little time put into whitespace and capitalization pays off. A few tips to help make your queries more readable:**

* If a select statement has more than one column, put each on a new line, indented from the select statement.
* Always capitalize SQL function names and keywords
* Put each clause of your query on a new line.
* Use indenting to make subqueries appear logically separate.

Another important consideration when writing readable queries is the use of **alias names and shortcuts**. Name aliases should be clear– a common convention is using the first letter of the table name, however if you feel that a query is complex you should consider using more explicit aliases.

`If you work in a team, you might consider a` [SQL style guide](http://www.sqlstyle.guide/)`— a great guide is available at SQL style guide, but remember that readability is more important than consistency. If you have a complex query and you think breaking the style guide will make it more readable, you should do it.`

# 3. The With Clause

When constructing complex queries, it's useful to create an intermediate table to produce our final results. You can use subqueries to create these intermediate tables. Unfortunately, the way subqueries are written makes it harder to read— the person reading the query needs to find the subquery and read from the inside-out.

`One way to alleviate this is to use a with clause`. WITH clauses allow you to define one or more named subqueries before the start of the main query. The main query then refers to the subquery by it's alias name, just as if it's a table in the database.

The syntax for the WITH clause is relatively straight-forward.

`WITH [alias_name] AS ([subquery])
SELECT [main_query]`

## TODO:
* Create a query that shows summary data for every playlist in the Chinook database:
* Use a WITH clause to create a named subquery with the following info:
  * The unique ID for the playlist.
  * The name of the playlist.
  * The name of each track from the playlist.
  * The length of each track in seconds.
* Your final table should have the following columns, in order:
  * playlist_id - the unique ID for the playlist.
  * playlist_name - The name of the playlist.
  * number_of_tracks - A count of the number of tracks in the playlist.
  * length_seconds - The sum of the length of the playlist in seconds.
* The results should be sorted by playlist_id in ascending order.

In [1]:
%load_ext sql

%sql sqlite:///chinook.db

In [2]:
%%sql

WITH playlist_info AS
    (
     SELECT
         p.playlist_id,
         p.name playlist_name,
         t.name track_name,
         (t.milliseconds / 1000) length_seconds
     FROM playlist p
     LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
     LEFT JOIN track t ON t.track_id = pt.track_id
    )

SELECT
    playlist_id,
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(length_seconds) length_seconds
FROM playlist_info
GROUP BY 1, 2
ORDER BY 1;

 * sqlite:///chinook.db
Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,876049
2,Movies,0,None
3,TV Shows,213,500987
4,Audiobooks,0,None
5,90’s Music,1477,397970
6,Audiobooks,0,None
7,Movies,0,None
8,Music,3290,876049
9,Music Videos,1,294
10,TV Shows,213,500987


# 4. Creating Views

When we use the WITH clause, we're creating a temporary named subquery that we can use only within that query. But what if we find ourselves using the same WITH with lots of different queries? It would be nice to permanently define a subquery that we can use again and again.

We do this by creating a view, which we can then use in all future queries. An easy way to think of this is the WITH clause creates a temporary view. The syntax for creating a view

`CREATE VIEW database.view_name AS
    SELECT * FROM database.table;`

## TODO:
* Create a view called customer_gt_90_dollars:
  * The view should contain the columns from customers, in their original order.
  * The view should contain only customers who have purchased more than $90 in tracks from the store.
* After the SQL query that creates the view, write a second query to display your newly created view: SELECT * FROM chinook.customer_gt_90_dollars;.
   * Make sure you use a semicolon (;) to indicate the end of each que

In [3]:
%%sql 

CREATE VIEW chinook.customer_gt_90_dollars AS 
    SELECT
        c.*
    FROM chinook.invoice i
    INNER JOIN chinook.customer c ON i.customer_id = c.customer_id
    GROUP BY 1
    HAVING SUM(i.total) > 90;
SELECT * FROM chinook.customer_gt_90_dollars;

 * sqlite:///chinook.db
(sqlite3.OperationalError) unknown database chinook
[SQL: CREATE VIEW chinook.customer_gt_90_dollars AS 
    SELECT
        c.*
    FROM chinook.invoice i
    INNER JOIN chinook.customer c ON i.customer_id = c.customer_id
    GROUP BY 1
    HAVING SUM(i.total) > 90;]
(Background on this error at: http://sqlalche.me/e/e3q8)


# 5. Combining Rows With Union

We have now created two views: customer_usa and customer_gt_90_dollars. How can we find customers who are in different permutations of these two views:

* Customers in the USA **or have** spent more than $90

* Customers in the USA **and have** spent more than $90

* Customers in the USA **and have** not spent more than $90

Where regular joins are used to join columns, the **union operator** is used to join rows from tables and/or views.



`[select_statement_one]
UNION
[select_statement_two]`

## TODO:
Use UNION to produce a table of customers in the USA or have spent more than $90, using the customer_usa and customer_gt_90_dollars views:
  * The result should contain the columns from customers, in their original order.

In [4]:
%%sql

SELECT * from customer_usa

UNION

SELECT * from customer_gt_90_dollars;

 * sqlite:///chinook.db
(sqlite3.OperationalError) no such table: customer_gt_90_dollars
[SQL: SELECT * from customer_usa

UNION

SELECT * from customer_gt_90_dollars;]
(Background on this error at: http://sqlalche.me/e/e3q8)


# 6. Combining Rows Using Intersect and Except

## TODO:
Write a query that works out how many customers that are in the USA and have purchased more than $90 are assigned to each sales support agent. For the purposes of this exercise, no two employees have the same name.

* Your result should have the following columns, in order:
* mployee_name - The first_name and last_name of the employee separated by a space, eg Luke Skywalker.
* customers_usa_gt_90 - The number of customer assigned to that employee that are both from the USA and have have purchased more than $90 worth of tracks.
* The result should include all employees with the title "Sales Support Agent", but not employees with any other title.
* Order your results by the employee_name column.

In [5]:
%%sql

WITH customers_usa_gt_90 AS
    (
     SELECT * FROM customer_usa

     INTERSECT

     SELECT * FROM customer_gt_90_dollars
    )

SELECT
    e.first_name || " " || e.last_name employee_name,
    COUNT(c.customer_id) customers_usa_gt_90
FROM employee e
LEFT JOIN customers_usa_gt_90 c ON c.support_rep_id = e.employee_id
WHERE e.title = 'Sales Support Agent'
GROUP BY 1 ORDER BY 1;

 * sqlite:///chinook.db
(sqlite3.OperationalError) no such table: customer_gt_90_dollars
[SQL: WITH customers_usa_gt_90 AS (
     SELECT * FROM customer_usa

     INTERSECT

     SELECT * FROM customer_gt_90_dollars
    )

SELECT
    e.first_name || " " || e.last_name employee_name,
    COUNT(c.customer_id) customers_usa_gt_90
FROM employee e
LEFT JOIN customers_usa_gt_90 c ON c.support_rep_id = e.employee_id
WHERE e.title = 'Sales Support Agent'
GROUP BY 1 ORDER BY 1;]
(Background on this error at: http://sqlalche.me/e/e3q8)


# 7. Multiple Named Subqueries

When we learned about WITH, we said with clauses allow you to define one or more named subqueries, but we didn't show you the syntax for creating more than one named subquery. To do this, you use a single WITH clause and multiple, comma-separated alias/subquery pairs:

`WITH
    [alias_name] AS ([subquery]),
    [alias_name_2] AS ([subquery_2]),
    [alias_name_3] AS ([subquery_3])
​
SELECT [main_query]`

## TODO:
Write a query that uses multiple named subqueries in a WITH clause to gather total sales data on customers from India:
* The first named subquery should return all customers that are from India.
* The second named subquery should calculate the sum total for every customer.
* The main query should join the two named subqueries, resulting in the following final columns:
   * customer_name - The first_name and last_name of the customer, separated by a space, eg Luke Skywalker.
   * total_purchases - The total amount spent on purchases by that customer.
* The results should be sorted by the customer_name column in alphabetical order.

In [6]:
%%sql

WITH
    customers_india AS
        (
        SELECT * FROM customer
        WHERE country = "India"
        ),
    sales_per_customer AS
        (
         SELECT
             customer_id,
             SUM(total) total
         FROM invoice
         GROUP BY 1
        )

SELECT
    ci.first_name || " " || ci.last_name customer_name,
    spc.total total_purchases
FROM customers_india ci
INNER JOIN sales_per_customer spc ON ci.customer_id = spc.customer_id
ORDER BY 1;

 * sqlite:///chinook.db
Done.


customer_name,total_purchases
Manoj Pareek,111.86999999999999
Puja Srivastava,71.28


# 8. Challenge: Each Country's Best Customer

Create a query to find the customer from each country that has spent the most money at our store, ordered alphabetically by country. Your query should return the following columns, in order:
* country - The name of each country that we have a customer from.
* customer_name - The first_name and last_name of the customer from that country with the most total purchases, separated by a space, eg Luke Skywalker.
* total_purchased - The total dollar amount that customer has purchased

In [7]:
%%sql

WITH
    customer_country_purchases AS
        (
         SELECT
             i.customer_id,
             c.country,
             SUM(i.total) total_purchases
         FROM invoice i
         INNER JOIN customer c ON i.customer_id = c.customer_id
         GROUP BY 1, 2
        ),
    country_max_purchase AS
        (
         SELECT
             country,
             MAX(total_purchases) max_purchase
         FROM customer_country_purchases
         GROUP BY 1
        ),
    country_best_customer AS
        (
         SELECT
            cmp.country,
            cmp.max_purchase,
            (
             SELECT ccp.customer_id
             FROM customer_country_purchases ccp
             WHERE ccp.country = cmp.country AND cmp.max_purchase = ccp.total_purchases
            ) customer_id
         FROM country_max_purchase cmp
        )
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer_name,
    cbc.max_purchase total_purchased
FROM customer c
INNER JOIN country_best_customer cbc ON cbc.customer_id = c.customer_id
ORDER BY 1 ASC

 * sqlite:///chinook.db
Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.38999999999999
Brazil,Luís Gonçalves,108.89999999999998
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02000000000001
Czech Republic,František Wichterlová,144.54000000000002
Denmark,Kara Nielsen,37.61999999999999
Finland,Terhi Hämäläinen,79.2


Well done on completing the challenge - that was a tough one! In this mission we've:

* continued to practice working with complex joins to answer queries
* learned how to use formatting and whitespace to make our queries more readable
* learned how to use WITH and VIEW to make temporary and permanent views
* learned how to perform set operations in SQL using UNION, INTERSECT, and EXCEPT